# Indexing and Selecting data


---

## Learning Objectives 


- Select data by position using `.isel()` with values or slices
- Select data by coordinate label/value using `.sel()` with values or slices
- Use nearest-neighbor lookups with `.sel()`
- Use `interp()` to interpolate by coordinate labels

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Understanding of xarray core data structures](some-link-to-external-resource) | Necessary | |
| [Basic familiarity with NumPy indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) | Helpful | |
| [Basic familiarity with Pandas indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) | Helpful | |

- **Time to learn**: *medium*



---

## Imports


In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset("./data/HadISST_sst.nc", engine="netcdf4")

## Different choices for indexing 


Xarray supports two kinds of indexing 

- Positional indexing via `.isel()`: provides primarily integer position based index (from `0` to `length-1` of the axis/dimension
- Label indexing via `.sel()`: provides primarily label based index



### Selection by position

The `.isel()` method is the primary access method for **purely integer based indexing**. The following are valid inputs:
- An integer e.g. `latitude=10`
- A list or array of integers `longitude=[10, 20, 39]`
- A slice object with integers e.g. `time=slice(2, 20)`

In [ ]:
sst = ds.sst
sst

In [ ]:
sst.isel()  # is the data as the original object i.e. no selection

In [ ]:
sst.isel(latitude=100)  # Equivalent to sst.isel(**{'latitude': 100})

In [ ]:
sst.isel(**{'latitude': 100})

In [ ]:
sst.isel(latitude=100, time=[-2, -1])

In [ ]:
sst.isel(longitude=100, time=slice(10, 20))

### Selection by label 


The `.sel()` method is the primary access method for **purely coordinate label based indexing.**. The following are valid inputs:

- A single coordinate label e.g. `time="2021-03-01"`
- A list or array of coordinate labels `longitude=[="2021-01-01", ="2021-03-10", ="2021-03-12"]`
- A slice object with coordinate labels e.g. `time=slice("2021-01-01", "2021-03-01")`.  (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index!)

In [ ]:
sst.sel(time="2021")

In [ ]:
sst.sel(time=slice("2020-01-01", "2020-12-31"))


<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning: Be careful when working with floating coordinate labels</p>
        
    When we have integer or string values for coordinate labels, "sel()" works flawlessly. When we try to work with floating coordinate labels, things get a little tricky:
        
</div>




In [ ]:
sst.sel(latitude=89.5)

In [ ]:
sst.sel(latitude=89.50001)

### Nearest-neighbor lookups

As shown above, when our coordinate labels are not integers or strings but floating point numbers, `.sel()` may throw a `KeyError`:

In [ ]:
sst.latitude

`sst.sel(latitude=89.50001)` fails because we are trying to use a conditional for an approximate value i.e floating numbers are represented approximately inside the computer, and xarray is unable to locate this exact value. To address this issue, xarray supports `method` and `tolerance` keyword argument. The `method` parameter allows for enabling nearest neighbor (inexact) lookups by use of the methods `'pad', 'backfill' or 'nearest'`: 

In [ ]:
sst.sel(latitude=[82.50001, 90.3689], method='nearest')

### Interpolation

If we want to interpolate along coordinates rather than looking up the nearest neighbos, we can use the `.interp()` method. To use `interp()` requires the presence of `scipy` library. 


In [ ]:
sst.interp(latitude=[10, 10.1, 10.2], method='nearest')

---

## Resources and References

- [Xarray Documentation - Indexing and Selecting Data](https://xarray.pydata.org/en/stable/indexing.html)
- [Xarray Documentation - Interpolation](https://xarray.pydata.org/en/stable/user-guide/interpolation.html)


<div class="admonition alert alert-success">
    <p class="title" style="font-weight:bold">Previous: <a href="./01-xarray-fundamentals.ipynb">Xarray Fundamentals</a></p>
         <p class="title" style="font-weight:bold">Next: <a href="./03-data-visualization.ipynb">Data Visualization</a></p>
</div>